In [1]:
import re
import pandas as pd
import numpy as np
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome(r'C:\chromedriver')
driver.get(r'http://www.ictjob.ph/search')
t1 = 7
njobs = int(driver.find_element_by_class_name('nb-jobs-found').text)
# pg_end = int(np.ceil(njobs/20))
pg_end = 1
pg = 0

failed = []
while (pg < pg_end):

    pg += 1
    sleep(t1+3)
    urls = [element.get_attribute('href').strip() for element in driver.find_elements_by_css_selector('.job-title.search-item-link')]
    titles = [element.text.strip() for element in driver.find_elements_by_css_selector('.job-title.search-item-link')]
    k_words = [element.text.strip() for element in driver.find_elements_by_class_name('job-keywords-card-section')]
    location = [element.text.strip() for element in driver.find_elements_by_class_name('job-location-card-section')]
    companies = [element.text.strip() for element in driver.find_elements_by_class_name('job-company-card-section')]

    tmp_l = [titles, companies, location, k_words, urls]
    len_chk = all(map(lambda x: len(x)==len(urls), tmp_l))

    if len_chk:
        jobs = list(zip(titles, companies, location, k_words, urls))
        df = pd.DataFrame(jobs, columns=['title','company','location','k_words','url'])
        df.to_csv(fr'ICTjobs/{pg}.csv', index=False)
    else:
        print('not the same length, skipping..')
        failed.append(pg)

    driver.find_element_by_class_name('next').click()

In [3]:
# Enter summary csv

df = pd.read_csv(r'ICTjobs/1.csv', usecols=['url'])
url_list = [url[0] for url in df.values.tolist()]
t1 = 7
jobd_list = []
for link in url_list:
    driver = webdriver.Chrome(r'C:\chromedriver')
    driver.get(link)
    sleep(t1)
    try:
        job_d = driver.find_element_by_class_name('contentpera').text
        li_list = [element.text for element in driver.find_element_by_class_name('contentpera').find_elements_by_tag_name('li')]
        jobd_list.append((link, job_d, li_list))
        driver.close()
    except:
        print(f'skipping url: {link}')
        driver.close()
        continue

df = pd.DataFrame(jobd_list, columns=['url','description', 'bullets'])
df.to_csv('ICTjobs/desc/desc-sum.csv', index=False)

In [4]:
print(len(jobd_list))

117


In [3]:
link

'http://www.ictjob.ph/search/job/l1-support-analyst/2591'

In [15]:
df = pd.read_csv('ICTjobs/1.csv')

In [16]:
df1 = pd.read_csv('ICTjobs/desc/desc-sum.csv')

In [17]:
df1 = df1.set_index('url')
df = df.set_index('url')
df_f = df1.join(df, how='inner')
df_f = df_f.reset_index()
df_f = df_f[['title', 'company', 'location', 'description', 'bullets', 'k_words', 'url']]
df_f.to_csv(r'ICTjobs/ICTjobs-Summary_new.csv')